In [5]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 加载模型和分词器
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 输入文本
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# 设置生成参数（移除 attention_mask 参数）
output = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    temperature=0.7,
    top_k=50,
    top_p=0.95
)

# 解码并打印生成的文本
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


/Users/chenhongyan/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ValueError: Greedy methods without beam search do not support `num_return_sequences` different than 1 (got 5).

In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.cuda.amp import autocast

# 加载模型和分词器
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 手动设置 pad_token_id，如果模型没有定义它
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# 输入文本
input_text = "Once upon a time in a dystopian future"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# 使用混合精度加速生成（如果使用GPU）
with autocast():
    output = model.generate(
        input_ids,
        max_length=100,  # 增加最大长度以生成更长的文本
        num_return_sequences=3,  # 一次生成 3 个序列进行比较
        pad_token_id=tokenizer.pad_token_id,
        temperature=0.8,  # 调整随机性
        top_k=50,  # 限制候选词数量
        top_p=0.9,  # 使用Nucleus采样
        repetition_penalty=1.2,  # 惩罚重复
        num_beams=5,  # 使用 Beam Search 增强生成质量
        early_stopping=True,  # 当满足停止条件时提前停止
        eos_token_id=tokenizer.convert_tokens_to_ids(".")  # 生成句号时停止
    )

# 解码并打印所有生成的文本
for i, generated_sequence in enumerate(output):
    generated_text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
    print(f"Generated Text {i + 1}: {generated_text}\n")


Generated Text 1: Once upon a time in a dystopian future, there was a man who had no idea what he was talking about.

Generated Text 2: Once upon a time in a dystopian future, there was a man who had the power to change the course of history.

Generated Text 3: Once upon a time in a dystopian future, there was a man who had no idea what he was doing.



In [10]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextGenerationPipeline
from torch.cuda.amp import autocast

# 加载模型和分词器
model_name = "gpt2-large"  # 使用更大的模型
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 使用TextGenerationPipeline进行批量生成，适合超大规模生成
generator = TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

batch_input_texts = [
    "In the near future, humans and machines coexist in harmony,",
    "The year is 3020, and space exploration has reached new heights.",
    "With the rise of quantum computing,",
    "In an alternate reality where time travel is possible,",
    "The discovery of a new planet has sparked international interest."
]

# 批量生成，使用Beam Search
batch_output = generator(batch_input_texts, max_length=150, num_return_sequences=5, 
                         temperature=0.7, top_k=50, top_p=0.95, repetition_penalty=1.2,
                         num_beams=5)  # 使用Beam Search，并设置num_beams

# 打印批量生成的结果
for i, outputs in enumerate(batch_output):
    for j, output in enumerate(outputs):
        print(f"Prompt {i + 1}, Sequence {j + 1}: {output['generated_text']}\n")


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt 1, Sequence 1: In the near future, humans and machines coexist in harmony, but that doesn't mean we won't have to work hard to make sure that doesn't happen.

This article was originally published on The Conversation. Read the original article. Follow all of the Expert Voices issues and debates — and become part of the discussion — on Facebook, Twitter and Google +. The views expressed are those of the author and do not necessarily reflect the views of the publisher. This version of the article was originally published on Live Science.

Prompt 1, Sequence 2: In the near future, humans and machines coexist in harmony, but that doesn't mean we won't have to work hard to make sure that doesn't happen.

This article was originally published on The Conversation. Read the original article. Follow all of the Expert Voices issues and debates — and become part of the discussion — on Facebook, Twitter and Google +. The views expressed are those of the author and do not necessarily reflect

In [12]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# 加载GPT-2模型和分词器
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义生成文本的函数
def generate_article(prompt, max_length=1000):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.7,  # 控制生成文本的随机性
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,  # 避免重复的n-gram
        num_return_sequences=1
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 提示文本
prompt = "写一篇关于人工智能对未来世界影响的简短文章："

# 生成文章
article = generate_article(prompt, max_length=100)

# 打印生成的文章
print("生成的文章：")
print(article)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


生成的文章：
写一篇关于人工智能对未来世界影响的简短文章：。
The first step is to find the right place for you. The second one will be a bit more difficult, but it's not impossible! You can also use your own skills and experience in this area as well:

 (1) Find an appropriate location


In [17]:
import torch
from transformers import GPT2LMHeadModel, BertTokenizer

# 加载中文的GPT-2模型和分词器
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义生成文本的函数
def generate_article(prompt, max_length=1000):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.7,  # 控制生成文本的随机性
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,  # 避免重复的n-gram
        num_return_sequences=1
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# 提示文本
prompt = "写一篇关于人工智能对未来世界影响的简短文章："

# 生成文章
article = generate_article(prompt, max_length=1000)

# 打印生成的文章
print("生成的文章：")
print(article)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


生成的文章：
写 一 篇 关 于 人 工 智 能 对 未 来 世 界 影 响 的 简 短 文 章 ： 人 类 的 未 知 ， 是 由 于 自 然 选 择 的 失 误 。 人 们 在 进 化 过 程 中 ， 不 断 地 寻 找 新 的 生 命 体 ， 而 这 些 新 生 物 ， 正 是 我 们 所 需 要 的 。 《 人 猿 》 作 者 ： 刘 晓 博 出 版 社 ： 北 京 大 学 出 品 人 ： 王 小 波 译 者 简 介 ： 李 开 复 ， 美 国 著 名 科 幻 作 家 、 科 技 评 论 家 ， 曾 任 职 于 美 联 储 主 席 耶 伦 ， 现 为 美 股 市 场 研 究 员 。 他 的 作 品 被 称 为 人 性 的 弱 点 ， 也 被 认 为 是 科 学 与 艺 术 之 间 的 差 异 。 李 先 生 是 一 位 有 着 丰 富 经 验 和 深 厚 理 想 的 科 普 作 曲 家 。 在 他 看 来 ， 人 的 思 维 方 式 是 无 法 改 变 的 ， 因 此 ， 他 希 望 通 过 自 己 的 创 造 力 ， 让 更 多 的 人 了 解 人 ， 并 且 帮 助 他 们 成 长 。 作 为 一 个 科 研 人 员 ， 李 老 师 的 观 点 非 常 独 到 ， 从 他 对 人 脑 的 理 解 ， 到 对 科 教 领 域 的 探 索 ， 都 值 得 我 去 思 考 。 作 文 题 目 ： 《 你 的 梦 想 是 什 么 ？ 》 （ 作 ） 作 用 ： 引 导 读 者 从 心 理 学 角 度 分 析 人 与 人 之 交 往 的 本 质 ， 以 及 如 何 利 用 人 际 关 系 ， 建 立 人 格 。 文 中 提 到 的 你 是 指 人 在 面 临 困 境 时 ， 会 采 取 行 动 的 那 种 人 。 这 种 行 为 可 以 说 是 人 生 的 一 部 分 ， 它 包 括 了 人 对 事 情 的 态 度 ， 或 者 是 对 自 身 的 认 识 。 但 是 ， 这 样 的 行 径 却 是 不 合 适 的 ： 人 总 是 在 不 停 地 试 图 寻 求 解 决 问 题 的 办 法 ， 甚 至 是 逃 避 痛 苦 。 因 为 ， 当 你 发 现 自 已 的 存 在 感 受 到 了 压 力 时 就 会 产 生 焦 虑 ， 觉 得 自 我 的 价 值 不 够 高

In [20]:
import torch
from transformers import GPT2LMHeadModel, BertTokenizer
import re

# 加载中文的GPT-2模型和分词器
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义生成文本的函数
def generate_article(prompt, max_length=10000):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.7,  # 控制生成文本的随机性
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,  # 避免重复的n-gram
        num_return_sequences=1
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return format_article(text)

# 定义文本格式化的函数
def format_article(text):
    # 根据20个字添加换行符
    formatted_text = "\n".join([text[i:i+80] for i in range(0, len(text), 20)])
    
    # 简单的标点符号修饰：在每个句子结尾添加句号
    formatted_text = re.sub(r'([。！？])', r'\1\n', formatted_text)  # 在句号、感叹号、问号后换行
    formatted_text = re.sub(r'([^。！？])$', r'\1。', formatted_text)  # 如果结尾没有标点，添加句号
    return formatted_text

# 提示文本
prompt = "写一篇关于人工智能对未来世界影响的简短文章："

# 生成文章
article = generate_article(prompt, max_length=1000)

# 打印生成的文章
print("生成的文章：")
print(article)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


生成的文章：
写 一 篇 关 于 人 工 智 能 对 未 来 世 界 影 响 的 简 短 文 章 ： 人 类 的 未 知 ， 是 由 于 自 然 选 择 的 失 误 。
 人 
未 来 世 界 影 响 的 简 短 文 章 ： 人 类 的 未 知 ， 是 由 于 自 然 选 择 的 失 误 。
 人 们 在 进 化 过 程 中 ， 不 断 
章 ： 人 类 的 未 知 ， 是 由 于 自 然 选 择 的 失 误 。
 人 们 在 进 化 过 程 中 ， 不 断 地 寻 找 新 的 生 命 体 ， 而 
于 自 然 选 择 的 失 误 。
 人 们 在 进 化 过 程 中 ， 不 断 地 寻 找 新 的 生 命 体 ， 而 这 些 新 生 物 ， 正 是 我 们 
们 在 进 化 过 程 中 ， 不 断 地 寻 找 新 的 生 命 体 ， 而 这 些 新 生 物 ， 正 是 我 们 所 需 要 的 。
 《 人 猿 》 作 
地 寻 找 新 的 生 命 体 ， 而 这 些 新 生 物 ， 正 是 我 们 所 需 要 的 。
 《 人 猿 》 作 者 ： 刘 晓 博 出 版 社 ： 北 
这 些 新 生 物 ， 正 是 我 们 所 需 要 的 。
 《 人 猿 》 作 者 ： 刘 晓 博 出 版 社 ： 北 京 大 学 出 品 人 ： 王 小 波 
所 需 要 的 。
 《 人 猿 》 作 者 ： 刘 晓 博 出 版 社 ： 北 京 大 学 出 品 人 ： 王 小 波 译 者 简 介 ： 李 开 复 ， 美 
者 ： 刘 晓 博 出 版 社 ： 北 京 大 学 出 品 人 ： 王 小 波 译 者 简 介 ： 李 开 复 ， 美 国 著 名 科 幻 作 家 、 科 技 
京 大 学 出 品 人 ： 王 小 波 译 者 简 介 ： 李 开 复 ， 美 国 著 名 科 幻 作 家 、 科 技 评 论 家 ， 曾 任 职 于 美 联 
译 者 简 介 ： 李 开 复 ， 美 国 著 名 科 幻 作 家 、 科 技 评 论 家 ， 曾 任 职 于 美 联 储 主 席 耶 伦 ， 现 为 美 股 
国 著 名 科 幻 作 家 、 科 技 评 论 家 ， 曾 任 职 于 美 联 储 主 席 耶 伦 ， 现 为 美 股 市 场 研 究 员 。
 他 的 作 品 
评 论 家 ， 曾 任 

In [22]:
import torch
from transformers import GPT2LMHeadModel, BertTokenizer
import jieba
import re

# 加载中文的GPT-2模型和分词器
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义生成文本的函数
def generate_article(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.7,  # 控制生成文本的随机性
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,  # 避免重复的n-gram
        num_return_sequences=1
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return format_article(text)

# 定义文本格式化的函数
def format_article(text):
    # 使用jieba进行分词
    words = jieba.lcut(text)
    
    # 动态换行：每行尽量保证一个完整的短句（5到20个字符之间）
    formatted_lines = []
    current_line = ""
    
    for word in words:
        current_line += word
        if len(current_line) >= 20 or (len(current_line) >= 5 and re.match(r'[。！？]', word)):
            formatted_lines.append(current_line)
            current_line = ""
    
    if current_line:
        formatted_lines.append(current_line)
    
    # 标点符号处理
    formatted_text = "\n".join(formatted_lines)
    formatted_text = re.sub(r'([^。！？])$', r'\1。', formatted_text)  # 如果结尾没有标点，添加句号
    
    return formatted_text

# 提示文本
prompt = "写一篇关于人工智能对未来世界影响的简短文章："

# 生成文章
article = generate_article(prompt, max_length=200)

# 打印生成的文章
print("生成的文章：")
print(article)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/mm/885dzr5d6s37b6tphkxb241c0000gn/T/jieba.cache
Loading model cost 0.565 seconds.
Prefix dict has been built successfully.


生成的文章：
写 一 篇 关 于 人 工 智 能 对 
未 来 世 界 影 响 的 简 短 文 
章 ： 人 类 的 未 知 ， 是 由 
于 自 然 选 择 的 失 误 。
 人 们 在 进 化 过 程 中 ， 不
 断 地 寻 找 新 的 生 命 体 ，
 而 这 些 新 生 物 ， 正 是 我
 们 所 需 要 的 。
 《 人 猿 》 作 者 ： 刘 晓 博
 出 版 社 ： 北 京 大 学 出 品
 人 ： 王 小 波 译 者 简 介 ：
 李 开 复 ， 美 国 著 名 科 幻
 作 家 、 科 技 评 论 家 ， 曾
 任 职 于 美 联 储 主 席 耶 伦
 ， 现 为 美 股 市 场 研 究 员
 。 他 的 作 品 被 称 为 人 性
 的 弱 点 ， 也 被 认 为 是 科
 学 与 艺 术 之 间 的 差 异 。
 李 先 生 是 一 位 有 着 丰 富
 经 验 和 深 厚 理 想。


In [25]:
import torch
from transformers import GPT2LMHeadModel, BertTokenizer
import jieba
import re
import spacy

# 加载中文的GPT-2模型和分词器
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# 加载中文的NLP模型，并添加sentencizer组件
nlp = spacy.blank("zh")
nlp.add_pipe('sentencizer')

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义生成文本的函数
def generate_article(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.7,  # 控制生成文本的随机性
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,  # 避免重复的n-gram
        num_return_sequences=1
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return format_article(text)

# 定义文本格式化的函数
def format_article(text):
    # 使用jieba进行分词
    words = jieba.lcut(text)
    
    # 使用spacy进行句子分割和语法分析
    doc = nlp(" ".join(words))
    
    # 动态换行：每行尽量保证一个完整的短句（5到20个字符之间）
    formatted_lines = []
    current_line = ""
    
    for sent in doc.sents:
        current_line += "".join([token.text for token in sent])
        if len(current_line) >= 20 or (len(current_line) >= 5 and re.match(r'[。！？]', current_line[-1])):
            formatted_lines.append(current_line)
            current_line = ""
    
    if current_line:
        formatted_lines.append(current_line)
    
    # 标点符号处理
    formatted_text = "\n".join(formatted_lines)
    formatted_text = re.sub(r'([^。！？])$', r'\1。', formatted_text)  # 如果结尾没有标点，添加句号
    
    return formatted_text

# 提示文本
prompt = "写一篇关于人工智能对未来世界影响的简短文章："

# 生成文章
article = generate_article(prompt, max_length=200)

# 打印生成的文章
print("生成的文章：")
print(article)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


生成的文章：
写  一  篇  关  于  人  工  智  能  对  未  来  世  界  影  响  的  简  短  文  章  ：  人  类  的  未  知  ，  是  由  于  自  然  选  择  的  失  误  。
  人  们  在  进  化  过  程  中  ，  不  断  地  寻  找  新  的  生  命  体  ，  而  这  些  新  生  物  ，  正  是  我  们  所  需  要  的  。
  《  人  猿  》  作  者  ：  刘  晓  博  出  版  社  ：  北  京  大  学  出  品  人  ：  王  小  波  译  者  简  介  ：  李  开  复  ，  美  国  著  名  科  幻  作  家  、  科  技  评  论  家  ，  曾  任  职  于  美  联  储  主  席  耶  伦  ，  现  为  美  股  市  场  研  究  员  。
  他  的  作  品  被  称  为  人  性  的  弱  点  ，  也  被  认  为  是  科  学  与  艺  术  之  间  的  差  异  。
  李  先  生  是  一  位  有  着  丰  富  经  验  和  深  厚  理  想。


In [27]:
import torch
from transformers import GPT2LMHeadModel, BertTokenizer
import jieba
import re

# 加载中文的GPT-2模型和分词器
gpt_model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(gpt_model_name)
tokenizer = BertTokenizer.from_pretrained(gpt_model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义生成文本的函数
def generate_article(prompt, max_length=200):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        num_return_sequences=1
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return format_article(text)

# 高级文本格式化的函数
def format_article(text):
    # 使用jieba分词
    words = jieba.lcut(text)
    formatted_text = ""

    current_line = ""
    for word in words:
        current_line += word
        if len(current_line) >= 20:
            formatted_text += current_line + "\n"
            current_line = ""

    if current_line:
        formatted_text += current_line

    # 处理标点符号
    formatted_text = re.sub(r'([^。！？])$', r'\1。', formatted_text)

    return formatted_text

# 提示文本
prompt = "写一篇关于人工智能对未来世界影响的简短文章："

# 生成文章
article = generate_article(prompt, max_length=400)

# 打印生成的文章
print("生成的文章：")
print(article)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


生成的文章：
写 一 篇 关 于 人 工 智 能 对 
未 来 世 界 影 响 的 简 短 文 
章 ： 人 类 的 未 知 ， 是 由 
于 自 然 选 择 的 失 误 。 人 
们 在 进 化 过 程 中 ， 不 断 
地 寻 找 新 的 生 命 体 ， 而 
这 些 新 生 物 ， 正 是 我 们 
所 需 要 的 。 《 人 猿 》 作 
者 ： 刘 晓 博 出 版 社 ： 北 
京 大 学 出 品 人 ： 王 小 波 
译 者 简 介 ： 李 开 复 ， 美 
国 著 名 科 幻 作 家 、 科 技 
评 论 家 ， 曾 任 职 于 美 联 
储 主 席 耶 伦 ， 现 为 美 股 
市 场 研 究 员 。 他 的 作 品 
被 称 为 人 性 的 弱 点 ， 也 
被 认 为 是 科 学 与 艺 术 之 
间 的 差 异 。 李 先 生 是 一 
位 有 着 丰 富 经 验 和 深 厚 
理 想 的 科 普 作 曲 家 。 在 
他 看 来 ， 人 的 思 维 方 式 
是 无 法 改 变 的 ， 因 此 ， 
他 希 望 通 过 自 己 的 创 造 
力 ， 让 更 多 的 人 了 解 人 
， 并 且 帮 助 他 们 成 长 。 
作 为 一 个 科 研 人 员 ， 李 
老 师 的 观 点 非 常 独 到 ， 
从 他 对 人 脑 的 理 解 ， 到 
对 科 教 领 域 的 探 索 ， 都 
值 得 我 去 思 考 。 作 文 题 
目 ： 《 你 的 梦 想 是 什 么 
？ 》 （ 作 ） 作 用 ： 引 导 
读 者 从 心 理 学 角 度 分 析 
人 与 人 之 交 往 的 本 质 ， 
以 及 如 何 利 用 人 际 关 系 
， 建 立 人 格 。 文 中 提 到 
的 你 是 指 人 在 面 临 困 境 
时 ， 会 采 取 行 动 的 那 种 
人 。 这 种 行 为 可 以 说。


In [32]:
import torch
from transformers import GPT2LMHeadModel, BertTokenizer
import jieba
import re

# 加載中文的GPT-2模型和分詞器
gpt_model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(gpt_model_name)
tokenizer = BertTokenizer.from_pretrained(gpt_model_name)

# 將模型移動到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定義生成文本的函數
def generate_article(prompt, max_length=200):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id  # 設置pad_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return format_article(text)

# 高級文本格式化的函數
def format_article(text):
    # 使用jieba分詞
    words = jieba.lcut(text)
    formatted_text = ""

    current_line = ""
    for word in words:
        current_line += word
        if len(current_line.replace(" ", "")) >= 20:  # 修正字符處理邏輯，確保沒有多餘空格
            formatted_text += current_line.strip() + "\n"
            current_line = ""

    if current_line:
        formatted_text += current_line.strip()  # 修正最後一行的處理

    # 處理標點符號
    formatted_text = re.sub(r'([^。！？])$', r'\1。', formatted_text)

    return formatted_text

# 提示文本
prompt = "寫一篇關於人工智能對未來世界影響的簡短文章："

# 生成文章
article = generate_article(prompt, max_length=400)

# 打印生成的文章
print("生成的文章：")
print(article)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


生成的文章：
寫 一 篇 關 於 人 工 智 能 對 未 來 世 界 影 響 的 簡 短 文
章 ： 、 、 ， 這 是 我 們 在 中 國 的 一 個 小 小 的 城 市
。 我 们 的 大 學 生 活 也 很 豐 富 ， 但 是 有 些 東 西 還
不 夠 豐 盛 。 比 如 說 ， 我 想 要 做 一 个 人 造 的 机 器
人 ， 可 以 把 我 的 手 臂 放 到 地 上 ， 然 後 用 它 来 控
制 我 自 己 的 身 体 。 但 我 覺 得 這 樣 做 太 過 於 繁 琐
了 ， 因 為 我 需 要 一 台 機 器 來 幫 助 我 去 完 成 任 務
。 所 以 我 希 望 能 够 讓 我 父 母 和 孩 子 都 能 感 受 到
人 類 的 快 樂 。 我 觉 得 这 样 的 话 ， 就 像 我 在 看 《
星 际 穿 越 》 时 候 说 的 那 样 ， 人 类 的 生 命 已 经 被
物 理 学 家 们 证 明 了 。 而 且 ， 他 们 还 提 出 了 一 种
新 的 方 法 ， 让 我 可 能 会 去 研 究 人 的 行 为 。 这 种
方 式 ， 对 于 我 来 说 非 常 好 。 因 为 我 知 道 ， 在 我
看 来 ， 这 是 一 件 非 同 寻 常 的 事 情 。 你 可 知 ， 当
你 看 到 一 群 人 在 一 起 玩 耍 的 时 间 ， 你 就 会 发 现
， 原 来 人 们 并 没 有 真 正 意 义 上 的 人 。 人 与 人 之
间 的 关 系 ， 其 实 是 相 互 依 存 的 。 在 这 里 ， 每 个
角 色 都 是 独 立 的 ， 只 是 他 的 位 置 不 同 罢 了 ！ （
本 文 由 135 编 辑 器 提 供 技 术 支 持 ） 丨 李 晓 峰
编 审 丨 张 志 强 资 料 来 源 丨 网 络。


In [36]:
import torch
from transformers import GPT2LMHeadModel, BertTokenizer
import jieba
import re

# 加载GPT-2模型和分词器
gpt_model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(gpt_model_name)
tokenizer = BertTokenizer.from_pretrained(gpt_model_name)

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义生成文本的函数
def generate_paragraph(prompt, max_length=600):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return format_paragraph(text)

# 定义格式化段落的函数
def format_paragraph(text):
    words = jieba.lcut(text)
    formatted_text = ""
    current_line = ""
    for word in words:
        current_line += word
        if len(current_line.replace(" ", "")) >= 20:
            formatted_text += current_line.strip() + "\n"
            current_line = ""

    if current_line:
        formatted_text += current_line.strip()

    return formatted_text

# 生成完整文章的函数
def generate_full_article(prompt, target_length=10000, paragraph_length=600):
    article = ""
    while len(article.replace(" ", "")) < target_length:
        paragraph = generate_paragraph(prompt, max_length=paragraph_length)
        article += paragraph + "\n\n"
        prompt = paragraph[-50:]  # 使用前一段落的最后部分作为下一段落的提示
    return article

# 提示文本
prompt = "写一篇关于人工智能对未来世界影响的深入分析文章："

# 生成10,000字的文章
article = generate_full_article(prompt, target_length=10000, paragraph_length=600)

# 打印生成的文章
print("生成的文章：")
print(article)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

生成的文章：
写 一 篇 关 于 人 工 智 能 对 未 来 世 界 影 响 的 深 入 分
析 文 章 ： 、 、 ， 这 是 我 在 问 答 社 区 上 看 到 的 一
个 问 题 。 我 想 说 的 是 ， 人 类 的 智 慧 可 以 用 数 学
模 型 去 解 释 ， 但 是 如 果 把 它 们 当 做 一 种 技 术 ，
那 么 就 会 变 成 一 件 很 难 的 事 情 。 因 为 人 们 不 知
道 自 己 有 多 少 智 力 ， 而 且 他 们 也 没 有 办 法 理 解
和 记 忆 。 所 以 ， 我 们 只 能 通 过 数 字 化 的 方 式 来
进 行 思 考 。 比 如 说 ， 你 可 能 会 发 现 ， 一 些 人 可
不 可 怕 ， 可 是 你 却 无 法 预 测 ， 甚 至 连 你 自 身 都
无 从 判 断 。 这 样 的 话 ， 就 像 你 的 孩 子 可 怜 巴 巴
地 听 着 妈 妈 讲 故 事 ， 然 后 你 就 可 悲 了 。 其 实 ，
大 家 都 知 识 太 匮 乏 ， 所 谓 的 智 商 高 低 并 非 真 正
意 义 上 的 聪 明 ， 更 多 的 还 是 要 靠 自 觉 性 。 如 何
让 自 已 的 脑 子 里 面 的 东 西 变 得 更 加 丰 富 ？ 我 认
为 ， 首 先 要 搞 清 楚 什 么 叫 智 障 。 智 者 ， 即 使 是
智 人 ， 也 需 要 有 一 定 的 逻 辑 思 维 能 力 。 而 这 种
逻 ， 又 是 怎 么 回 事 呢 ？ 就 是 说 ： 智 取 威 虎 山 ，
不 仅 仅 是 为 了 赢 得 胜 利 ， 还 要 为 自 我 的 生 存 提
供 保 证 。 那 就 好 比 说 一 句 话 ： 我 不 是 傻 瓜 ， 只
是 笨 蛋 。 这 句 ， 是 一 位 智 的 哲 学 观 点 。 他 认 同
聪 才 的 概 念 ， 认 识 到 了 智 的 本 质 ， 并 且 认 定 了
愚 蠢 二 字 。 聪 的 人 往 往 具 备 超 越 常 人 的 天 赋 ，
他 的 思 想 是 不 断 地 创 造 新 的 价 值 。 但 智 则 万 物
， 智 也 万 象 ， 聪 也 是 人 之 常 情 ， 愚 也 就 不 再 是
单 纯 的 意 志 力 了 ， 它 是 由 人 体 内 的 某 些

In [37]:
import torch
from transformers import GPT2LMHeadModel, BertTokenizer, pipeline
import jieba
import re

# 加载GPT-2模型和分词器
gpt_model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(gpt_model_name)
tokenizer = BertTokenizer.from_pretrained(gpt_model_name)

# 使用pipeline加载生成管道，指定设备为GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# 定义生成段落的函数，支持多次生成候选段落
def generate_paragraph(prompt, max_length=600, num_return_sequences=5):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id
    )
    candidates = [tokenizer.decode(o, skip_special_tokens=True) for o in output]
    # 使用BERTScore或其他质量评估方法选择最佳候选段落
    best_candidate = max(candidates, key=lambda x: len(x))  # 简单选择最长的段落
    return format_paragraph(best_candidate)

# 定义格式化段落的函数
def format_paragraph(text):
    words = jieba.lcut(text)
    formatted_text = ""
    current_line = ""
    for word in words:
        current_line += word
        if len(current_line.replace(" ", "")) >= 20:
            formatted_text += current_line.strip() + "\n"
            current_line = ""

    if current_line:
        formatted_text += current_line.strip()

    return formatted_text

# 生成完整文章的函数
def generate_full_article(prompt, target_length=10000, paragraph_length=600):
    article = ""
    while len(article.replace(" ", "")) < target_length:
        paragraph = generate_paragraph(prompt, max_length=paragraph_length)
        article += paragraph + "\n\n"
        prompt = paragraph[-50:]  # 使用前一段落的最后部分作为下一段落的提示
    return article

# 提示文本
prompt = "写一篇关于人工智能对未来世界影响的深入分析文章："

# 生成10,000字的文章
article = generate_full_article(prompt, target_length=10000, paragraph_length=600)

# 打印生成的文章
print("生成的文章：")
print(article)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

生成的文章：
写 一 篇 关 于 人 工 智 能 对 未 来 世 界 影 响 的 深 入 分
析 文 章 ： ？ 本 人 是 一 个 技 术 学 院 毕 业 生 ， 现 在
在 做 人 机 交 互 相 关 方 面 的 实 验 。 很 早 就 开 始 接
触 人 类 与 机 器 之 间 的 沟 通 技 巧 了 ， 比 如 机 械 手
和 数 据 传 递 ， 这 些 技 能 我 都 有 过 涉 猎 ， 但 是 并
没 有 深 刻 研 究 。 我 们 做 过 一 次 机 会 谈 话 后 ， 我
给 出 了 一 份 简 单 的 答 案 ： 未 必 所 有 人 都 喜 欢 你
（ 或 者 说 不 会 讨 厌 你 ） ， 而 是 需 要 和 你 讨 论 的
事 情 可 以 被 理 解 、 能 够 应 用 到 自 己 身 上 。 因 为
在 进 行 实 际 生 活 中 ， 你 无 法 获 得 真 正 的 信 息 ，
同 时 也 没 办 法 确 保 你 知 道 任 何 人 和 物 体 的 状 态
。 这 种 状 况 下 ， 人 们 就 可 能 把 它 称 作 人 的 思 维
过 程 中 所 产 生 的 问 题 。 有 一 句 名 言 叫 做 我 只 会
打 字 ， 别 人 却 能 看 见 我 的 样 子 ， 当 然 还 有 另 外
一 种 情 况 的 人 ， 他 们 根 本 就 不 懂 人 怎 么 理 性 地
处 理 这 件 事 。 举 例 来 说 ， 一 群 年 轻 人 就 像 一 条
狗 一 样 ， 从 小 训 练 他 去 学 习 逻 辑 。 他 的 主 要 目
标 是 让 他 成 为 一 名 合 格 的 数 学 家 。 最 初 ， 数 字
化 计 算 机 可 行 吗 ？ 我 认 为 很 难 ， 甚 至 不 可 想 象
。 但 在 某 些 领 域 里 ， 它 能 做 的 更 好 。 在 几 十 万
个 智 力 测 试 中 找 到 方 法 论 就 足 够 了 。 然 而 ， 如
果 要 做 到 这 点 ， 则 需 花 费 大 量 的 时 间 和 精 力 。
比 方 说 google deep learning
（ 即 人 脑 神 经 网 络 ） 里 的 一 些 人 在 开 发 一 款 新
的 ai 软 件 。 如 今 ， 许 多 人 对 ai 有 着 极 强 的
兴 趣 ， 认 

In [45]:
import torch
from transformers import BertTokenizer, GPT2LMHeadModel
import jieba
import opencc

# 使用Hugging Face提供的标准中文GPT-2模型
model_name = "uer/gpt2-chinese-cluecorpussmall"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained("uer/gpt2-chinese-cluecorpussmall")

# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 创建繁简转换器
cc = opencc.OpenCC('s2t')  # s2t 是 Simplified to Traditional 的简写

# 定义生成段落的函数
def generate_paragraph(prompt, max_length=600):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = input_ids.ne(tokenizer.pad_token_id)  # 设置 attention_mask
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        no_repeat_ngram_size=2,
        pad_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    traditional_text = cc.convert(text)  # 转换为繁体中文
    return format_paragraph(traditional_text)

# 定义格式化段落的函数
def format_paragraph(text):
    words = jieba.lcut(text)
    formatted_text = ""
    current_line = ""
    for word in words:
        current_line += word
        if len(current_line.replace(" ", "")) >= 20:
            formatted_text += current_line.strip() + "\n"
            current_line = ""

    if current_line:
        formatted_text += current_line.strip()

    return formatted_text

# 生成完整文章的函数
def generate_full_article(prompt, target_length=10000, paragraph_length=600):
    article = ""
    while len(article.replace(" ", "")) < target_length:
        paragraph = generate_paragraph(prompt, max_length=paragraph_length)
        article += paragraph + "\n\n"
        prompt = paragraph[-50:]  # 使用前一段落的最后部分作为下一段落的提示
    return article

# 提示文本
prompt = "寫一篇關於外星人的幻想文章："

# 生成10,000字的文章
article = generate_full_article(prompt, target_length=10000, paragraph_length=600)

# 将文章保存到文件
with open("generated_article_traditional.txt", "w", encoding="utf-8") as file:
    file.write(article)

# 打印生成的文章
print("生成的文章已保存到 generated_article_traditional.txt 文件中")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

生成的文章已保存到 generated_article_traditional.txt 文件中
